In [ ]:
import sys
import warnings

import numpy
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import metrics
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

import time

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import os

# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']

In [ ]:
# Cluster Datasets:
# Cluster 1: November, December, January, February
# Cluster 2: June, July, August
# Cluster 3: April, May, September, October, March

### Cluster 1

In [ ]:
# Cluster Datasets:
# Cluster 1: November, December, January, February

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']

# Filter DataFrame for the months November, December, January, February
df = df[df.index.month.isin([11, 12, 1, 2])]

# Print or use the resulting DataFrame
print(df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Assuming you have a time series data stored in a pandas DataFrame called 'data'
# with a column named 'value' containing the values

# Convert the 'value' column to a numpy array
# values = data['value'].to_numpy()

# Plot ACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_acf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF)')
plt.show()

# Plot PACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_pacf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
df_train=df['2017-04-01':'2020-03-31']
df_test=df['2020-04-01':]
df_train.shape, df_test.shape

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:

datasets=df.values
X, Y = create_dataset(datasets, 8)

train_size = df_train.shape[0]
test_size =  df_test.shape[0]
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]

#  Random Forest Regressor
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = StandardScaler()
sc_y = StandardScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(max_features=6)

rfr.fit(X,y)
y_pred_train_rfr= rfr.predict(X)
y_pred_test_rfr= rfr.predict(X1)

y_pred_train_rfr=pd.DataFrame(y_pred_train_rfr)
y_pred_test_rfr=pd.DataFrame(y_pred_test_rfr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_rfr= sc_y.inverse_transform (y_pred_test_rfr)
y_pred_train1_rfr=sc_y.inverse_transform (y_pred_train_rfr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_rfr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_rfr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_rfr_c1 = mean_absolute_percentage_error(y_train,y_pred_train1_rfr)
rmse_train_rfr_c1 = sqrt(mean_squared_error(y_train,y_pred_train1_rfr))
mae_train_rfr_c1 = metrics.mean_absolute_error(y_train,y_pred_train1_rfr)

# summarize the fit of the model on test data
mape_test_rfr_c1 = mean_absolute_percentage_error(y_test,y_pred_test1_rfr)
rmse_test_rfr_c1 = sqrt(mean_squared_error(y_test,y_pred_test1_rfr))
mae_test_rfr_c1 = metrics.mean_absolute_error(y_test,y_pred_test1_rfr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_rfr_c1)
print(rmse_train_rfr_c1)
print(mae_train_rfr_c1)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_rfr_c1)
print(rmse_test_rfr_c1)
print(mae_test_rfr_c1)

rfr_time=time.time() - start_time
print("--- %s seconds -#  Random Forest Regressor---" % (rfr_time))


### Cluster 2

In [ ]:
# Cluster 2: June, July, August

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']

# Assuming you have already loaded and filtered your DataFrame df
# If not, you can use the provided code to load and filter it

# Filter DataFrame for the months 'June', 'July', 'August'
df = df[df.index.month.isin([6, 7, 8])]

# Print or use the resulting DataFrame
print(df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Assuming you have a time series data stored in a pandas DataFrame called 'data'
# with a column named 'value' containing the values

# Convert the 'value' column to a numpy array
# values = data['value'].to_numpy()

# Plot ACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_acf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF)')
plt.show()

# Plot PACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_pacf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
df_train=df['2017-04-01':'2020-03-31']
df_test=df['2020-04-01':]
df_train.shape, df_test.shape

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
datasets=df.values
X, Y = create_dataset(datasets, 8)

train_size = df_train.shape[0]
test_size =  df_test.shape[0]
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]

#  Random Forest Regressor
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = StandardScaler()
sc_y = StandardScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(max_features=4, max_depth=5, n_estimators=50 )

rfr.fit(X,y)
y_pred_train_rfr= rfr.predict(X)
y_pred_test_rfr= rfr.predict(X1)

y_pred_train_rfr=pd.DataFrame(y_pred_train_rfr)
y_pred_test_rfr=pd.DataFrame(y_pred_test_rfr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_rfr= sc_y.inverse_transform (y_pred_test_rfr)
y_pred_train1_rfr=sc_y.inverse_transform (y_pred_train_rfr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_rfr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_rfr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_rfr_c2 = mean_absolute_percentage_error(y_train,y_pred_train1_rfr)
rmse_train_rfr_c2 = sqrt(mean_squared_error(y_train,y_pred_train1_rfr))
mae_train_rfr_c2 = metrics.mean_absolute_error(y_train,y_pred_train1_rfr)

# summarize the fit of the model on test data
mape_test_rfr_c2 = mean_absolute_percentage_error(y_test,y_pred_test1_rfr)
rmse_test_rfr_c2 = sqrt(mean_squared_error(y_test,y_pred_test1_rfr))
mae_test_rfr_c2 = metrics.mean_absolute_error(y_test,y_pred_test1_rfr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_rfr_c2)
print(rmse_train_rfr_c2)
print(mae_train_rfr_c2)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_rfr_c2)
print(rmse_test_rfr_c2)
print(mae_test_rfr_c2)

rfr_time=time.time() - start_time
print("--- %s seconds -#  Random Forest Regressor---" % (rfr_time))


### Cluster 3

In [ ]:
# Cluster 3: April, May, September, October, March

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']


# Filter DataFrame for the months April, May, September, October, March

df = df[df.index.month.isin([3, 4, 5, 9, 10])]

# Print or use the resulting DataFrame
print(df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Assuming you have a time series data stored in a pandas DataFrame called 'data'
# with a column named 'value' containing the values

# Convert the 'value' column to a numpy array
# values = data['value'].to_numpy()

# Plot ACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_acf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF)')
plt.show()

# Plot PACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_pacf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
df_train=df['2017-04-01':'2020-03-31']
df_test=df['2020-04-01':]
df_train.shape, df_test.shape

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:

datasets=df.values
X, Y = create_dataset(datasets, 9)

train_size = df_train.shape[0]
test_size =  df_test.shape[0]
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]

#  Random Forest Regressor
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = StandardScaler()
sc_y = StandardScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(max_features=6)

rfr.fit(X,y)
y_pred_train_rfr= rfr.predict(X)
y_pred_test_rfr= rfr.predict(X1)

y_pred_train_rfr=pd.DataFrame(y_pred_train_rfr)
y_pred_test_rfr=pd.DataFrame(y_pred_test_rfr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_rfr= sc_y.inverse_transform (y_pred_test_rfr)
y_pred_train1_rfr=sc_y.inverse_transform (y_pred_train_rfr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_rfr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_rfr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_rfr_c3 = mean_absolute_percentage_error(y_train,y_pred_train1_rfr)
rmse_train_rfr_c3 = sqrt(mean_squared_error(y_train,y_pred_train1_rfr))
mae_train_rfr_c3 = metrics.mean_absolute_error(y_train,y_pred_train1_rfr)

# summarize the fit of the model on test data
mape_test_rfr_c3 = mean_absolute_percentage_error(y_test,y_pred_test1_rfr)
rmse_test_rfr_c3 = sqrt(mean_squared_error(y_test,y_pred_test1_rfr))
mae_test_rfr_c3 = metrics.mean_absolute_error(y_test,y_pred_test1_rfr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_rfr_c3)
print(rmse_train_rfr_c3)
print(mae_train_rfr_c3)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_rfr_c3)
print(rmse_test_rfr_c3)
print(mae_test_rfr_c3)

rfr_time=time.time() - start_time
print("--- %s seconds -#  Random Forest Regressor---" % (rfr_time))


# whole

In [ ]:
# Load your original dataset
df = pd.read_excel(r"UP08.xlsx", index_col='Date_Time')
df = df[['dissolvedoxygenmeasured']]
df=df['2017-04-01':'2021-03-31']
df

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Assuming you have a time series data stored in a pandas DataFrame called 'data'
# with a column named 'value' containing the values

# Convert the 'value' column to a numpy array
# values = data['value'].to_numpy()

# Plot ACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_acf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF)')
plt.show()

# Plot PACF
fig, ax = plt.subplots(figsize=(10, 3))
plot_pacf(df['dissolvedoxygenmeasured'][:100], ax=ax,alpha=0.05)
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
# convert an array of values into a dataset matrix
import numpy 
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100


In [ ]:
# Split the data into training and testing sets
train_data = df.loc['2017-04-01':'2020-03-31']
test_data = df.loc['2020-04-01':]

In [ ]:
train_data.shape, test_data.shape

In [ ]:

datasets=df.values
X, Y = create_dataset(datasets, 5)

train_size = train_data.shape[0]
test_size =  test_data.shape[0]
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]

#  Random Forest Regressor
start_time = time.time()

numpy.random.seed(1234)
tf.random.set_seed(1234)

X_train=pd.DataFrame(trainX)
Y_train=pd.DataFrame(trainY)
X_test=pd.DataFrame(testX)
Y_test=pd.DataFrame(testY)

sc_X = StandardScaler()
sc_y = StandardScaler()

X= sc_X.fit_transform(X_train)
y= sc_y.fit_transform(Y_train)
X1= sc_X.transform(X_test)
y1= sc_y.transform(Y_test)
y=y.ravel()
y1=y1.ravel()  

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(max_features=5)

rfr.fit(X,y)
y_pred_train_rfr= rfr.predict(X)
y_pred_test_rfr= rfr.predict(X1)

y_pred_train_rfr=pd.DataFrame(y_pred_train_rfr)
y_pred_test_rfr=pd.DataFrame(y_pred_test_rfr)

y1=pd.DataFrame(y1)
y=pd.DataFrame(y)

y_pred_test1_rfr= sc_y.inverse_transform (y_pred_test_rfr)
y_pred_train1_rfr=sc_y.inverse_transform (y_pred_train_rfr)

y_test= sc_y.inverse_transform (y1)
y_train= sc_y.inverse_transform (y)

y_pred_test1_rf=pd.DataFrame(y_pred_test1_rfr)
y_pred_train1_rf=pd.DataFrame(y_pred_train1_rfr)

y_test= pd.DataFrame(y_test)


#summarize the fit of the model on train data
mape_train_rfr = mean_absolute_percentage_error(y_train,y_pred_train1_rfr)
rmse_train_rfr = sqrt(mean_squared_error(y_train,y_pred_train1_rfr))
mae_train_rfr = metrics.mean_absolute_error(y_train,y_pred_train1_rfr)

# summarize the fit of the model on test data
mape_test_rfr = mean_absolute_percentage_error(y_test,y_pred_test1_rfr)
rmse_test_rfr = sqrt(mean_squared_error(y_test,y_pred_test1_rfr))
mae_test_rfr = metrics.mean_absolute_error(y_test,y_pred_test1_rfr)

# train scores
print("The metrics for the training data are: ")
print(mape_train_rfr)
print(rmse_train_rfr)
print(mae_train_rfr)

# test scores
print("The metrics for the testing data are: ")
print(mape_test_rfr)
print(rmse_test_rfr)
print(mae_test_rfr)

rfr_time=time.time() - start_time
print("--- %s seconds -#  Random Forest Regressor---" % (rfr_time))



In [ ]:
# Comparison

In [ ]:
# using whole dataset vs clusters for prediction 
print(mape_test_rfr, mape_test_rfr_c1,mape_test_rfr_c2,mape_test_rfr_c3)
print(rmse_test_rfr, rmse_test_rfr_c1,rmse_test_rfr_c2,rmse_test_rfr_c3)
print(mae_test_rfr, mae_test_rfr_c1,mae_test_rfr_c2,mae_test_rfr_c3)


In [ ]:
# using whole dataset vs clusters for prediction 
print(mape_test_rfr, (((4/12)*mape_test_rfr_c1)+((3/12)*mape_test_rfr_c2)+((5/12)*mape_test_rfr_c3)))
print(rmse_test_rfr, (((4/12)*rmse_test_rfr_c1)+((3/12)*rmse_test_rfr_c2)+((5/12)*rmse_test_rfr_c3)))
print(mae_test_rfr, (((4/12)*mae_test_rfr_c1)+((3/12)*mae_test_rfr_c2)+((5/12)*mae_test_rfr_c3)))
